In [2]:
import pandas as pd
import geopandas as gpd
import string

In [3]:
df = pd.read_csv("2016_General.csv")

In [4]:
# Select all Race Names in 2016_General.csv
selected = df[(df.CounterType == "Times Counted")]
countdict = {}
for race in selected.Race:
    countdict.setdefault(race, 0)
    countdict[race] += 1
    
# Find out the number of precincts in the county,
# those that aren't "max" don't take place in every precinct
total_precincts = max(countdict.values())

# Select only countywide races
county_wide_races = [race for race, count in countdict.items() if count == total_precincts]

# Confusingly rename df, but only with countywide_races
only_countywide_races = df[df.Race.isin(county_wide_races)]

only_countywide_races = only_countywide_races.drop(columns = ["LEG", "CC", "CG", "CounterGroup", "Party"])


In [5]:
# Make new column that combines "Race" and "CounterType", then drop them
only_countywide_races["Race_Type"] = only_countywide_races["Race"] + ": " + only_countywide_races["CounterType"]
only_countywide_races = only_countywide_races.drop(columns = ["Race", "CounterType"])

#
only_countywide_races = only_countywide_races.pivot(index='Precinct', columns='Race_Type', values='SumOfCount')

# New Columns and drop others
only_countywide_races["Registered_Voters"] = only_countywide_races["Advisory Vote 14: Registered Voters"]
only_countywide_races["Total_Voters"] = only_countywide_races["Advisory Vote 14: Times Counted"]
only_countywide_races = only_countywide_races.drop(columns = [column for column in only_countywide_races.columns if "Registered Voters" in column]) 
only_countywide_races = only_countywide_races.drop(columns = [column for column in only_countywide_races.columns if "Times Counted" in column]) 


# Bring two created columns to beginning
cols = only_countywide_races.columns.tolist()
cols = cols[-2:] + cols[:-2]
only_countywide_races = only_countywide_races[cols]


In [6]:
# a function to shorten field names

def encode(n):
    ''' Convert integer n to string.ascii_lowercase'''
    if n == 0:
        return "a"
    out = []
    while n:
        n, r = n // 26, n % 26
        out.append(string.ascii_lowercase[r])
    return ''.join(out)

# Create a data_dict so we know what these field names mean
cols = only_countywide_races.columns.tolist()
data_dict = {column_name: encode(i) for i, column_name in enumerate(cols)}

with open("data_dictionary.csv", "w+") as fh:
    fh.write("O.G. value, encoded as\n")
    for k, v in data_dict.items():
        fh.write(f'{k}, {v}\n')


In [7]:
# Apply our new data_dict to the data, write to csv
only_countywide_races = only_countywide_races.rename(index = str, columns = data_dict)
only_countywide_races.to_csv("csv2016.csv")

In [15]:
#### below here is the auto join.


### Thing we have to worry about for our app -- normalization? based on que? More math in JS?!?! aw.
## alt. option would be to multiply answer by what will be the divisor in the future!? 
## But too hacky? also would mess up (numbers). Not a good idea. Sigh.

df = pd.read_csv("csv2016.csv")
spatial = gpd.read_file("KC_Precincts.geojson")


join_csv_to_geojson(csv_dict, geojson_dict):
    """
    Take a dict containing csvs, with key being file_name.csv 
    and value being "join on this value" 
    
    geojson_dict is a dict containing just one k:v :
    {"file_name.geojson": "join_on_this_value} 
    """
    geojson_file, geojson_join_field = list(geojson_dict.keys())[0]
    for csv_file, csv_join_field in csv_dict.items():
        pass
        # do stuff
        
#  = spatial.merge(country_names, on='iso_a3')

In [18]:
for column in spatial.columns.tolist():
    print(column)

NAME
geometry


In [11]:
### A section for writing index.html; all other files to folder
## build dropdown option menu
# for column_name, encoded_value in data_dict.items():
#     ballot_name = " ".join("".join(column_name.replace("&","").replace("_",":").split(":")[1:2]).split()[:3])
#     if "Voted" not in ballot_name: 
#         print(f'<option value="{encoded_value};{ballot_name}">{column_name}</option>')

